# Background settings

Load packages

In [1]:
import json
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import t, gaussian_kde

Plotting options

In [2]:
colors = '#2B8EE3', '#F9B43C', '#FA672F', '#BE1E2D', '#A7A9AC','#2B8EE3', '#F9B43C', '#FA672F', '#BE1E2D', '#A7A9AC'

plt.style.use('ggplot')
colors = [d['color'] for d in list(plt.rcParams['axes.prop_cycle'])]

if os.path.exists('font/'):    
    prop = fm.FontProperties(fname=f'font/micross.ttf')
    mpl.rcParams['font.family'] = prop.get_name()

Feature sets

In [3]:
feature_sets = json.loads(open('data/features_sets_info.json','r').read())
feature_sets_map = dict(zip(feature_sets['labels'], feature_sets['labels_exhaust']))
feature_sets_map_expand = dict(zip(feature_sets['labels_expand'], feature_sets['labels_exhaust_expand']))

Model selection and label 

In [21]:
model_select = dict(
    lr_p1 = "(clf=='LR')&(fs==True)&(poly_degree==1)&(bins=='3 - uneven')",
    lr_p2 = "(clf=='LR')&(fs==True)&(poly_degree==2)&(bins=='3 - uneven')",    
    rf_t_fs = "(clf=='RF')&(estimate_hyper_param==True)&(fs==True)",
    rf_t_nfs = "(clf=='RF')&(estimate_hyper_param==True)&(fs==False)")

model_map = dict(lr_p1=('Logistic Regression', 'polynom. order=1'),
                 lr_p2=('Logistic Regression', 'polynom. order=2'),           
                 rf_t_fs=('Random forest', 'with feat. select.'),
                 rf_t_nfs=('Random forest', 'w/o feat. select.'))

for max_d, mss in [(20,5),(10,5),(50,5),(20,2),(20,10)]:
    key = f'rf_nt_{max_d}md_{mss}mss'     
    model_select[key] = \
        f"(clf=='RF')&(estimate_hyper_param==False)&(clf_max_depth=={max_d})&(clf_min_samples_split=={mss})"
    model_map[key]=\
        'Random forest', f'fixed hyperparam. (max_depth,{max_d}; min_sample_split,{mss})'
    
for bins in 3,4,5:
    key = f'lr_p1_even{bins}'
    model_select[key] = f'bins=="{bins}"'
    model_map[key] = f'Log. regression', f'pol. ord.=1; {bins} even bin target' 

model_process = ['lr_p1','lr_p2','rf_nt_20md_5mss','rf_t_nfs', 'lr_p1_even3', 'lr_p1_even4']
models_plot_agg = {k:v for k,v in model_select.items() if k in model_process}

model_label_exhaust = {k:', '.join(v) for k,v in model_map.items()}

Model comparison information

In [25]:
# feature groupings
feature_groups = dict(
    major = ['combined_all','pre_all','post_all'],
    expanding = ['pre_all', 'pre_elem_and_elem','pre_elem'],
    task=['task', 'non_task'],
    all_feat_sets=feature_sets['labels'])

feature_group_letter = dict(major='A',expanding='C',task='B')

comparison_sets = dict(
    major = [('pre_all', 'post_all'), ('pre_all', 'combined_all')],
    expanding = [('pre_elem', 'pre_elem_and_elem'), ('pre_elem_and_elem', 'pre_all')],
    all_feat_sets = [],
    task=[('non_task','task')])

# Structure results 

#### Load model output

In [26]:
data_models = pd.read_csv('data/models_a.csv').append(pd.read_csv('data/models_b.csv'))
data_bins = pd.read_csv('data/bins.csv')

data = pd.concat([data_models, data_bins.query('bins!="3 - uneven"')],sort=False).copy()
data.bins = data.bins.fillna('3 - uneven')


data['feature_set_label_exhaust'] = \
    pd.Categorical(data.feature_set_label.map(feature_sets_map),
                   categories=feature_sets['labels_exhaust'],
                   ordered=True)
for acronym, selection in model_select.items():    
    data.loc[data.query(selection).index, 'Model'] = ' - '.join(model_map[acronym])

#### Generate table and plots

In [27]:
mean_acc = {}
comparison_df = {model_label_exhaust[v]:{} for v in models_plot_agg.keys()}
acc_dict = {}

models_plot = {k:v for k,v in model_select.items() if k in models_plot_agg.keys()}

for model_label, selection in models_plot.items():

    mle = model_label_exhaust[model_label]
    mean_acc[mle]=\
        data\
            .query(selection)\
            .groupby('feature_set_label_exhaust')\
            .accuracy\
            .mean()
    
    
    acc_gb_fs = \
        data\
        .query(selection)\
        .groupby('feature_set_label_exhaust').accuracy
    
    acc_fs_agg =acc_gb_fs.agg(['mean', 'std'])
    acc_fs_agg.index = \
        [s.replace('\n', ' ') for s in acc_fs_agg.index.tolist()]
    acc_dict[mle] = acc_fs_agg
    
    for feature_group, comparison_set_ in feature_groups.items():
        
        label = '%s_%s' % (feature_group, model_label)

        ####################
        # data structuring #
        ####################
        compare_selection = data.feature_set_label.isin(comparison_set_)        
        df = data.loc[compare_selection].query(selection).copy()

        # features
        fs_map = feature_sets_map
        fs_exhaust = feature_sets['labels_exhaust']
        if feature_group=='expanding':
            fs_map = feature_sets_map_expand
            fs_exhaust = feature_sets['labels_exhaust_expand']

        fs_exhaust_sub = df.feature_set_label.map(fs_map)

        df['fs_long'] = \
            pd.Categorical(fs_exhaust_sub,
                           categories=[f for f in fs_exhaust if f in fs_exhaust_sub.unique()],
                           ordered=True)
        df_idx = df.set_index('feature_set_label')

        # table agg
        for set1, set2 in comparison_sets[feature_group]:
            diff = df_idx.loc[set2].accuracy.values-df_idx.loc[set1].accuracy.values
            mu, std, corr = diff.mean(), diff.std(), np.sqrt(1/1000+1/3)
            t_corr = mu/(corr*std)
            p_corr = t.sf(np.abs(t_corr),1000-1)*2
            comparison_pair = '%s vs %s' % (feature_sets_map[set1], feature_sets_map[set2])
            cp = comparison_pair.replace('\n', ' ')
            comparison_df[mle][cp] = '%.2f (p=%.4f)'% (mu, p_corr)


        # figure output
        figpath = 'output/%s_letter.png' % label
        if not os.path.exists(figpath):
            has_x_label = feature_group == 'expanding'#in ('task', 'major')
            f,ax = plt.subplots(figsize=(6,0.8+0.6*len(comparison_set_)+0.2*has_x_label))

            sub = df.reset_index().pipe(lambda df: df[df.feature_set_label.isin(comparison_set_)])

            if feature_group == 'expanding':
                colors_ = sns.light_palette(colors[1],n_colors=4)[1:][::-1]        
            elif feature_group == 'task':
                colors_ = colors[3:]
            elif feature_group == 'major':
                colors_ = [colors[i] for i in  [2,1,0]]
            else:
                colors_ = colors

            # make KDE part
            n = len(comparison_set_)
            pos = list(range(1,n+1))
            accs = [sub[sub.feature_set_label==fs].accuracy.values for fs in comparison_set_]
            widths = [] 
            for fs in feature_groups[feature_group]:
                acc = df[df.feature_set_label==fs].accuracy
                kde = gaussian_kde(acc)
                widths.append(kde(acc).max()/12)
            parts = ax.violinplot(accs, pos, widths=widths, vert=False, showextrema=False) 
            ax.set_yticks(pos)
            ax.set_yticklabels([fs_map[fs] for fs in comparison_set_])
            for i, pc in enumerate(parts['bodies']):
                if feature_group!='all_feat':
                    pc.set_color(colors_[i])
                pc.set_edgecolor('black')
                pc.set_alpha(1)

            # make box plot part (quantiles)
            m,q05,q10,q25,q50,q75,q90,q95,M = np.quantile(accs,[0,.05,.1,.25,.5,.75,.9,.95,1],1)
            ax.hlines(pos, q10, q90, color='k', lw=1.5)
            ax.hlines(pos, q25, q75, color='k', lw=7)
            ax.scatter(q50,pos,color='white',marker='o',s=20,zorder=3)

            # shared
            ax.set_ylabel('')
            
            if model_label == 'lr_p1_even4':
                ax.axvline(x=0.25, color='gray',linestyle='--')
            elif model_label == 'lr_p1_even5':    
                ax.axvline(x=0.2, color='gray',linestyle='--')
            else:
                ax.axvline(x=0.3333, color='gray',linestyle='--')
                
            ax.set_xlim(0.2, 0.8)
            if has_x_label:
                ax.set_xlabel('Balanced accuracy')
                plt.subplots_adjust(bottom=.2)
            else:
                ax.set_xlabel('')
            ax.yaxis.grid(False)    

            plt.subplots_adjust(left=.24)

            if feature_group in feature_group_letter:
                ax.text(0.03,0.33+n, feature_group_letter[feature_group], size=24)

            f.savefig(figpath)
            f.savefig(figpath.replace('png','pdf'))

Generate plots of hyperparameters for logistic regression

In [15]:
for i, label in enumerate(feature_sets['labels'][:3]):    
    figpath = f'output/lr_hyperparams_{label}.png'
    if not os.path.exists(figpath):
        f,ax = plt.subplots(1,2,figsize=(12,4))
        ax[0].set_xlabel('Ridge parameter, log scaled (=$\log_{10}(\lambda_2))$')
        ax[1].set_xlabel('Number of features used')
        plotdata_ = data_models.query(f'(feature_set_label=="{label}")&(clf=="LR")&(poly_degree==1)')
        plotdata_\
            .loc[:,'hyperparam_clf__C']\
            .pipe(np.log10)\
            .plot\
            .hist(ax=ax[0],bins=20)
        plotdata_\
            .loc[:,'hyperparam_fs__max_features']\
            .value_counts()\
            .sort_index()\
            .plot(ax=ax[1])

        ax[1].set_xlim(1,31)

        f.suptitle(feature_sets['labels_exhaust'][i])
    
        f.savefig(figpath)
        f.savefig(figpath.replace('png','pdf'))

Tabulate accuracy across models and statistical tests

In [28]:
labels_alt_classifier = [model_label_exhaust[m] for m in ['lr_p1','lr_p2', 'rf_nt_20md_5mss','rf_t_nfs']]
labels_alt_target = [model_label_exhaust[m] for m in ['lr_p1', 'lr_p1_even3', 'lr_p1_even4']]

# statististical tests overview 
comparison_tab = pd.DataFrame(comparison_df)
comparison_tab[labels_alt_classifier].to_csv('output/comparison_tab_classifiers.csv')
comparison_tab[labels_alt_target].to_csv('output/comparison_tab_targets.csv')


# accuracy overview - different models
acc_tab = pd.concat(acc_dict,axis=1).round(3)
acc_tab.columns = \
    pd.MultiIndex.from_arrays(arrays=[acc_tab.columns.get_level_values(0),
                                      ['Mean', 'Std. Dev.']*len(acc_dict)],
                              names=['Model', 'Feature set'])
acc_tab[labels_alt_classifier].to_csv('output/acc_tab_classifiers.csv')
acc_tab[labels_alt_target].to_csv('output/acc_tab_targets.csv')
